In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread as skioread
from skimage import io as skio
import ssl
import urllib
import cv2
import gc
%matplotlib inline

In [ ]:
driver = webdriver.Chrome('/home/frohman/Desktop/driver/chromedriver')

In [ ]:

## 菊花
a = 'https://diag.tactri.gov.tw/Search/CropListPrevent?CropSelcId=2600&Page=1&Row=100'

## 非洲菊
b = 'https://diag.tactri.gov.tw/Search/CropListPrevent?CropSelcId=2580&Page=1&Row=100'


In [ ]:
target = a

In [ ]:
if target == a:
    driver.get(a)
else:
    driver.get(b)

In [ ]:
def getCsv(a,sp='./',sn='default'):
    o = {}
    o['name'] = []
    o['image'] = []
    o['url'] = []
    for tr in driver.find_elements_by_tag_name('tr'):
        tds = tr.find_elements_by_tag_name('td')
        if len(tds) > 0:
            imgs = tds[3].find_elements_by_tag_name('img')
            name = tds[2]
            url = tds[2].find_element_by_tag_name('a')
            if name!='' and len(imgs) > 0:
                for i in imgs:
                    o['name'].append(name.text)
                    o['image'].append(i.get_property('src'))
                    o['url'].append(url.get_property('href'))
    data = pd.DataFrame.from_dict(o)
    data.to_csv(sp+sn+".csv",index=False)
    return o

In [ ]:
def showImage(url):
    from skimage import io
    io.imshow(io.imread(url))
    io.show()

In [ ]:
# showImage("http://www.w3schools.com/css/trolltunga.jpg")

In [ ]:
if target == a:
    getCsv(driver,sn='data1')
else:
    getCsv(driver,sn='data2')

In [ ]:
driver.close()

In [ ]:
val = ['疫病','炭疽病','Rhizopus sp.引起之真菌性病害','萎凋病',
 '細菌性葉斑病',
 '非洲菊斑潛蠅',
 '鐮孢菌引起之萎凋病',
 '立枯病',
'猝倒病', 
'白絹病',
'白粉病', 
'Phoma sp.引起之真菌性病害',
 '蠅類',
'青枯病',
'Rhizoctonia sp.引起之立枯病', 
'薊馬類',
'莖腐病','白銹病'
]

In [ ]:
temp1 = pd.read_csv('./data1.csv')

In [ ]:
temp2 = pd.read_csv('./data2.csv')

In [ ]:
temp1.shape

In [ ]:
temp1[temp1['name'] == '']

In [ ]:
(temp1['name'].isin(val)).sum()

In [ ]:
temp2.shape

In [ ]:
temp2.head()

In [ ]:
(temp2['name'].isin(val)).sum()

In [ ]:
temp1['name'].unique()

In [ ]:
temp2['name'].unique()

In [ ]:
temp1 = temp1[temp1['name'].isin(val)]

In [ ]:
temp2 = temp2[temp2['name'].isin(val)]

In [ ]:
temp1.shape

In [ ]:
temp2.shape

In [ ]:
temp1.to_csv('./finaldata1.csv',index=False)

In [ ]:
temp2.to_csv('./finaldata2.csv',index=False)

In [ ]:
temp1 = pd.read_csv('./finaldata1.csv')

In [ ]:
temp2 = pd.read_csv('./finaldata2.csv')

In [ ]:
temp1.shape

In [ ]:
temp1.head()

In [ ]:
temp2.shape

In [ ]:
temp2.head()

In [ ]:
temp3 = pd.concat([temp1,temp2])

In [ ]:
temp3.to_csv('./concatfinal.csv',index=False)

In [ ]:
test = temp3['image']

In [2]:
def convertImage(url):
    context = ssl._create_unverified_context()
    try:
        resp =  urllib.request.urlopen(url, context=context)
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_ANYCOLOR)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    except:
        print(url)
        image = url
    return image

In [3]:
temp3 = pd.read_csv('./concatfinal.csv')

In [4]:
temp3 = temp3[temp3['image'] != 'https://diag.tactri.gov.tw/public/UploadDgnsResultImage/14698/5392f012-b0ba-48d2-9949-38f7bba91164.jpg']

In [5]:
z = temp3['image'].apply(convertImage)

https://diag.tactri.gov.tw/public/UploadDgnsResultImage/14698/1a306214-5176-4f52-bdb2-f52bf174fbaf.jpg
https://diag.tactri.gov.tw/public/UploadDgnsResultImage/14698/adb598e6-29f1-4ead-b0fb-f9d3a0e86144.jpg


In [6]:
temp3['image'] = z.tolist()

In [7]:
tt = temp3['image'].map(lambda x: type(x) != str)

In [8]:
temp4 = temp3[tt]

In [ ]:
import gc

In [ ]:
del z,temp3,tt

In [ ]:
gc.collect()

In [9]:
temp4.head()

,name,image,url
0,萎凋病,"[[[5, 8, 13], [6, 9, 14], [7, 10, 15], [7, 10,...",https://diag.tactri.gov.tw/Search/CropPrevDeta...
1,萎凋病,"[[[8, 9, 14], [8, 9, 14], [8, 9, 14], [7, 8, 1...",https://diag.tactri.gov.tw/Search/CropPrevDeta...
2,萎凋病,"[[[12, 10, 21], [11, 9, 20], [8, 6, 17], [7, 5...",https://diag.tactri.gov.tw/Search/CropPrevDeta...
3,立枯病,"[[[50, 54, 53], [51, 55, 54], [56, 60, 59], [4...",https://diag.tactri.gov.tw/Search/CropPrevDeta...
4,立枯病,"[[[54, 58, 67], [65, 69, 78], [85, 89, 100], [...",https://diag.tactri.gov.tw/Search/CropPrevDeta...


In [10]:
path = './'

In [11]:
import os

In [13]:
i=0
for img, n in zip(temp4['image'],temp4['name']):
    print(i)
    p = path+"data"+"/"+n
    if not os.path.exists(p):
        os.mkdir(p)
    skio.imsave(p+"/"+str(i)+".jpeg",img)
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184


In [14]:
del i,img,n,p

In [17]:
gc.collect()

4

In [18]:
temp4.shape

(185, 3)

In [ ]:
1+1

In [19]:
tt = temp4['image']

In [ ]:
temp4['image'] = tt.apply(lambda x: x.tolist())

In [ ]:
temp4.to_csv('./train.csv',index=False)